In [133]:
import random
import numpy as np

# Hospital-Doctor Assignment Problem

In [141]:
cap = 5
doc_n = 50
hosp_n = 10

In [148]:
mat2 = np.empty([doc_n,hosp_n])
for i in range(doc_n):
    mat2[i,:]=np.array(random.sample(range(hosp_n),hosp_n))  
mat2 = mat2 + np.ones((doc_n,hosp_n))

In [149]:
mat2

array([[ 4.,  9.,  8.,  6.,  1.,  3.,  2., 10.,  5.,  7.],
       [10.,  2.,  7.,  8.,  3.,  6.,  4.,  9.,  1.,  5.],
       [ 6.,  2.,  9., 10.,  4.,  3.,  8.,  5.,  1.,  7.],
       [ 4.,  6.,  7., 10.,  9.,  3.,  5.,  8.,  1.,  2.],
       [ 1.,  5.,  3.,  7.,  6.,  4.,  8.,  2.,  9., 10.],
       [ 5.,  2.,  7., 10.,  4.,  6.,  1.,  9.,  8.,  3.],
       [ 6.,  8.,  1.,  3.,  7., 10.,  9.,  4.,  2.,  5.],
       [ 5.,  9., 10.,  1.,  6.,  2.,  3.,  8.,  7.,  4.],
       [ 3.,  8., 10.,  1.,  9.,  2.,  6.,  4.,  5.,  7.],
       [ 7., 10.,  3.,  1.,  5.,  6.,  8.,  4.,  2.,  9.],
       [ 2.,  3.,  7.,  1.,  8., 10.,  5.,  4.,  9.,  6.],
       [ 1.,  3.,  6.,  8.,  5.,  7.,  4.,  2., 10.,  9.],
       [ 2.,  9.,  6.,  7.,  1.,  3.,  4.,  5., 10.,  8.],
       [10.,  3.,  6.,  9.,  1.,  2.,  4.,  5.,  7.,  8.],
       [ 4., 10.,  8.,  5.,  6.,  1.,  7.,  2.,  9.,  3.],
       [ 3.,  5.,  4.,  2.,  6.,  8., 10.,  9.,  1.,  7.],
       [ 5.,  7.,  2.,  9.,  1., 10.,  6.,  3.,  4.,  8.

We decided to start assigning doctors from the least preffered hospital, as defined as the hospital with the highest sum of rankings, ie the one that, when assigned, would make the most doctors unhappy. Once this hospital chosen, we assign to it the doctor who wants it the most (lowest rank, closest to 1). Since this doctor has now been assigned, we "remove" them from consideration in the matrix by assigning very large numbers to that row (so they will never be picked again - a doctor will always have a lower rank than the one just assigned). We take into account the new assignment and remove 1 from said hospital's capacity. 

We iterate this process until every doctor has a hospital. Every time, we check the capacity of the hospital that has just been picked as having the highest rank. If it is full, we "cancel out" this hospital from future consideration by assigning any unpicked doctors preference as a very negative number (so that the rank will never have a sum bigger than a hospital not at capacity, and thus not be picked in the future). 

By the end of the loop, every doctor is assigned to a hospital. We then filter out the indices that correspond to assignements, setting them as 1, and assign 0 elsewhere. This matrix now holds the correspondance between each doctor and their assigned hospital (represented by the indices in the matrix, simply). 

We are interested in how well assignments were performed. To do so, we multiply the matrix described above with the input matrix. This reveals the rank that each doctor had put for their assigned hospital. This allows us to check the score of the algorithm, by doing the sum. The closer this value is to the number of doctors times their number one rank (the optimal case, ie every doctor is assigned to their number one choice), the better the algorithm performed. 


In [150]:
def master_function(mat2, cap, doc_n, hosp_n):
    # mat2: matrix with each row representing a docotor, and each column a hospital, filled with the hopsital rankings
    # cap: the hopsitals' capacities (same for each one)
    # doc_n: the total number of doctors
    # hosp_n: the total number of hospitals
    
    # add a row with the capacities of each hospital at the end of the matrix
    capacities = np.full((1,hosp_n), cap) 
    mat2= np.append(mat2, capacities, axis=0)
    
    #make a copy of the matrix for future reference
    mat= mat2.copy()

    #before entering the loop, create initial values (choose least preffered hospital)
    mat3 = np.sum(mat2[:-1,:],0) 
    hos = np.argmax(mat3)
    num=0
    
    #loop over number of doctors
    while (num < doc_n):
        #check if the least preffered hospital is full
        if(mat2[-1,hos] == 0 ):
            # remove this hospital from further consideration
            mat2[:-1,hos] = np.where(mat2[:-1,hos] < (hosp_n+1), -(hosp_n+1), mat2[:-1,hos])
            #find the new least prefered hospital out of the remaining ones (not at capacity)
            mat3 = np.sum(mat2[:-1,:],0)
            #stores the indice of said hospital 
            hos = np.argmax(mat3)
        
        #if the hospital is not at capacity
        else:
            mat3 = np.sum(mat2[:-1,:],0)
            hos = np.argmax(mat3)
            mat2[-1,hos]-=1
            #assign to the hospital the doctor with the highest preference
            doc = np.argmin(mat2[:-1,hos])
            #once assigned, remove the doctor from further consideration
            mat2[doc,]=(hosp_n+doc_n/0.05 +1)
            mat2[doc,hos]= (hosp_n+1)
            num +=1
    
    #filter out of the matrix the indices that do not correspond to assigned doctors
    mat2 = np.where(mat2[:-1,:] > (hosp+1), 0, 1)
    #multiply this filtered matrix by the initial matrix
    te = mat2*mat[:-1,:]
    
    #calculate the score of the result obtained 
    score = np.sum(np.sum(te[:-1,:],0))
    
    #create an array with the correspondance of each doctor to each hospital
    results = np.empty((doc_n,2))
    results[:,0]= np.where(te!=0)[0]
    results[:,1]= np.where(te!=0)[1]
    
    #return the results, the score of the trial and the matrix of the correspondances with the initial ranks
    return results, score, te

In [151]:
results, score, te = master_function(mat2, cap, doc_n, hosp_n)

In [157]:
for i in range(1,11):
    print(i, np.sum(te==i))


1 38
2 8
3 2
4 2
5 0
6 0
7 0
8 0
9 0
10 0


In [153]:
score

67.0